<a href="https://colab.research.google.com/github/ShaswataJash/kfpcomponent/blob/main/DataPreparationUsingPycaret/Data_preparation_using_PyCaret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version

Python 3.7.13


In [ ]:
!pip install kfp==1.8.12

In [ ]:
!pip install pycaret==2.3.10

     |████████████████████████████████| 320 kB 5.2 MB/s 
     |████████████████████████████████| 2.0 MB 47.0 MB/s 
     |████████████████████████████████| 1.3 MB 49.2 MB/s 
     |████████████████████████████████| 17.8 MB 526 kB/s 
     |████████████████████████████████| 1.7 MB 42.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 56 kB 3.8 MB/s 
     |████████████████████████████████| 167 kB 46.3 MB/s 
     |████████████████████████████████| 6.8 MB 45.1 MB/s 
     |████████████████████████████████| 120 kB 71.9 MB/s 
     |████████████████████████████████| 88 kB 6.4 MB/s 
     |████████████████████████████████| 262 kB 56.9 MB/s 
     |████████████████████████████████| 690 kB 47.9 MB/s 
     |████████████████████████████████| 636 kB 58.5 MB/s 
     |████████████████████████████████| 4.7 MB 43.3 MB/s 
     |██████████████████

In [ ]:
#Ref: https://github.com/pycaret/pycaret/issues/2490
!pip install Jinja2==3.1.2

     |████████████████████████████████| 133 kB 5.0 MB/s 
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!curl https://rclone.org/install.sh | sudo bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4497  100  4497    0     0   8453      0 --:--:-- --:--:-- --:--:--  8453
Archive:  rclone-current-linux-amd64.zip
   creating: tmp_unzip_dir_for_rclone/rclone-v1.58.1-linux-amd64/
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.58.1-linux-amd64/rclone  [binary]
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.58.1-linux-amd64/README.html  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.58.1-linux-amd64/README.txt  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.58.1-linux-amd64/git-log.txt  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.58.1-linux-amd64/rclone.1  [text]  
Purging old database entries in /usr/share/man...
Processing manual pages under /usr/share/man...
Purging old database entries in /usr/share/man/ko...
Processing manual pages under /usr/share/man/ko...
Purging old database entries

In [ ]:
import os
os.environ["RCLONE_CONFIG_REMOTEREAD_TYPE"] = 'http'
os.environ["RCLONE_CONFIG_REMOTEREAD_URL"] = 'https://raw.githubusercontent.com/pycaret/datasets/main/data/common/'

## CREATE THE rclone read and write configuration file name as 'REMOTEREAD' and 'REMOTEWRITE'. Because the same are used within code.
## So convention for creating any environment variables related to rclone should start either with 'RCLONE_CONFIG_REMOTEREAD' or 'RCLONE_CONFIG_REMOTEWRITE'

#https://registry.opendata.aws/humor-detection/
#os.environ["RCLONE_CONFIG_REMOTEREAD_TYPE"] = 's3'
#os.environ["RCLONE_CONFIG_REMOTEREAD_PROVIDER"] = 'AWS'
#os.environ["RCLONE_CONFIG_REMOTEREAD_ACCESS_KEY_ID"] = ''
#os.environ["RCLONE_CONFIG_REMOTEREAD_SECRET_ACCESS_KEY"] = ''
#os.environ["RCLONE_CONFIG_REMOTEREAD_REGION"] = 'us-west-2' 
#os.environ["RCLONE_CONFIG_REMOTEREAD_ENDPOINT"] = ''

os.environ["RCLONE_CONFIG_REMOTEWRITE_TYPE"] = 'local'
os.environ["RCLONE_CONFIG_REMOTEWRITE_NOUNC"] = 'true'

#refer https://rclone.org/faq/#:~:text=Can%20I%20use%20rclone%20with,for%20services%20reached%20over%20https%20).
#os.environ["http_proxy"]=''
#os.environ["https_proxy"]=''
#os.environ["HTTP_PROXY"]=''
#os.environ["HTTPS_PROXY"]=''

In [ ]:
input_datasource_directory_mountable = False
#input_datasource_directory_to_be_mounted = 'humor-detection-pds/'
#input_datasource_file_name = 'Non-humours-biased.csv'

In [1]:
%%writefile data_preparation.py

#!/usr/bin/env python3
import argparse

# Defining and parsing the command-line arguments
parser = argparse.ArgumentParser(description='kubeflow pipeline component to read csv file and prepare the data')
parser.add_argument('--input-datasource-directory-mountable', default=False, action="store_true", help='whether input csv file is present in mountable remote location')
parser.add_argument('--input-datasource-directory-to-be-mounted', type=str, help='if input-datasource-directory-mountable=True, name of the mountable directory (e.g. bucket name for s3)')
parser.add_argument('--input-datasource-file-name', type=str, help='name of the csv file including file extension (if any)')
parser.add_argument('--additional-options-csv-parsing', type=str, default= '{}', help='json formatted key-value pairs of strings which will be passed to pandas.read_csv()')
parser.add_argument('--type-of-data-analysis-task', choices=['classification', 'regression', 'clustering', 'anomaly_detection'])
parser.add_argument('--target-variable-name', type=str, help='for classification and regression, specify the column name holding target variable')
parser.add_argument('--target-emptyindicator', type=str, default='', help='if target variable column holds null or na, those rows will be dropped. Sometime empty can be indicated by other representative string like - or *** etc')
parser.add_argument('--generate-panda-dataframe-profiling-html', default=False, action="store_true", help='whether to generate data-profile data for original and transformed dataframe (can be timeconsuming)')
parser.add_argument('--data-preparations-options', type=str, default= '{}', help='json formatted key-value pairs of strings which will be passed to pycaret setup() function')
parser.add_argument('--additional-options-csv-writing', type=str, default= '{}', help='json formatted key-value pairs of strings which will be passed to pandas.to_csv()')
parser.add_argument('--output-datasource-directory-to-be-mounted', type=str, help='name of the mountable directory (e.g. bucket name for s3)')
parser.add_argument('--output-datasource-file-name', type=str, help='filename of the prepared data')
args = parser.parse_args()

import tempfile
local_datastore_read_dir = tempfile.mkdtemp(prefix="my_local_read-")
print('local_datastore_read_dir:',local_datastore_read_dir)

local_datastore_write_dir = tempfile.mkdtemp(prefix="my_local_write-")
print('local_datastore_write_dir:',local_datastore_write_dir)

#input file handling
import subprocess
import os
import sys
if args.input_datasource_directory_mountable:
    input_data_read_cmd = "rclone -v mount remoteread:" + args.input_datasource_directory_to_be_mounted + ' ' + local_datastore_read_dir + ' --daemon'
else:
    input_data_read_cmd = "rclone -v copy remoteread:" + args.input_datasource_file_name + ' ' + local_datastore_read_dir
input_data_read_call = subprocess.run(input_data_read_cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
print(input_data_read_call.stdout)
if input_data_read_call.returncode != 0:
    print("Error in rclone, errorcode=", input_data_read_call.returncode)
    sys.stdout.flush()
    sys.exit("Forceful exit as rclone returned error in context of reading")

#output file handling
output_data_write_cmd = "rclone -v mount remotewrite:" + args.output_datasource_directory_to_be_mounted + ' ' + local_datastore_write_dir + ' --daemon'
output_data_write_call = subprocess.run(output_data_write_cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
print(output_data_write_call.stdout)
if output_data_write_call.returncode != 0:
    print("Error in rclone, errorcode=", output_data_write_call.returncode)
    sys.stdout.flush()
    sys.exit("Forceful exit as rclone returned error in context of writing")

#handling input csv file reading
import pandas
import json
from pandas_profiling import ProfileReport
try:
    parse_config = json.loads(args.additional_options_csv_parsing)
    print('parse_config = (', type(parse_config), ')', parse_config)
    parse_config['filepath_or_buffer'] = os.path.join(local_datastore_read_dir,args.input_datasource_file_name)
    my_data = pandas.read_csv(**parse_config)
    if args.generate_panda_dataframe_profiling_html:
        original_profile = ProfileReport(my_data, title="Original dataframe", explorative=True)
        original_profile.to_file("original_data_report.html")
    print(my_data)
    
except BaseException as err:
    print("Error=", err, ' ', type(err))
    sys.stdout.flush()
    sys.exit("Forceful exit as exception encountered while parsing input csv file")

#handling data preprocessing
import pycaret
try:
    if os.path.exists("logs.log"):
        os.remove("logs.log") #removing any content from log which pycaret will internally use for its own logging
    print('pycaret version = ', pycaret.utils.version())
    setup_config = json.loads(args.data_preparations_options)
    print('setup_config = (', type(setup_config), ')', setup_config)
    if args.type_of_data_analysis_task == 'classification':
        import pycaret.classification
        setup_fn = pycaret.classification.setup
        get_config_fn = pycaret.classification.get_config
        setup_config['target'] = args.target_variable_name
        
    elif args.type_of_data_analysis_task == 'regression':
        import pycaret.regression
        setup_fn = pycaret.regression.setup
        get_config_fn = pycaret.regression.get_config
        setup_config['target'] = args.target_variable_name

    elif args.type_of_data_analysis_task == 'clustering':
        import pycaret.clustering
        setup_fn = pycaret.clustering.setup
        get_config_fn = pycaret.clustering.get_config

    elif args.type_of_data_analysis_task == 'anomaly':
        import pycaret.anomaly
        setup_fn = pycaret.anomaly.setup
        get_config_fn = pycaret.anomaly.get_config
        
    #as part of pycaret's data cleaning the rows with target column = nan are not being cleaned up. Thus, cleaning those rows explicitely
    if len(args.target_emptyindicator) > 0:
        #ref: https://stackoverflow.com/questions/49291740/delete-rows-if-there-are-null-values-in-a-specific-column-in-pandas-dataframe
        import numpy as np
        my_data[args.target_variable_name] = my_data[args.target_variable_name].replace(args.target_emptyindicator, np.nan)
        my_data = my_data.dropna(axis=0, subset=[args.target_variable_name])

    setup_config['data'] = my_data
    setup_config['log_experiment'] = False
    setup_config['data_split_shuffle'] = False
    setup_config['html'] = False
    setup_config['silent'] = True
    setup_fn(**setup_config)
    #ref: https://www.kdnuggets.com/2020/11/5-things-doing-wrong-pycaret.html
    X_transformed = get_config_fn('X')
    my_transformed_data = X_transformed
    if args.type_of_data_analysis_task == 'classification' or args.type_of_data_analysis_task == 'regression':
        y_transformed = get_config_fn('y')
        my_transformed_data = X_transformed.merge(y_transformed,left_index=True, right_index=True)
    print(my_transformed_data)

    pycaret.utils.get_system_logs() #this will print the pycaret's own log into console

    if args.generate_panda_dataframe_profiling_html:
        transformed_profile = ProfileReport(my_transformed_data, title="Transformed dataframe", explorative=True)
        transformed_profile.to_file("transformed_data_report.html")
    
except BaseException as err:
    pycaret.utils.get_system_logs()
    print("Error=", err, ' ', type(err))
    sys.stdout.flush()
    sys.exit("Forceful exit as exception encountered while transforming input dataframe")

#handling output csv file writing
try:
    to_csv_config = json.loads(args.additional_options_csv_writing)
    print('to_csv_config = (', type(to_csv_config), ')', to_csv_config)
    to_csv_config['path_or_buf'] = os.path.join(local_datastore_write_dir,args.output_datasource_file_name)
    my_transformed_data.to_csv(**to_csv_config)
except BaseException as err:
    print("Error=", err, ' ', type(err))
    sys.stdout.flush()
    sys.exit("Forceful exit as exception encountered while trying to write prepared data")

Writing data_preparation.py


In [ ]:
!python data_preparation.py --input-datasource-file-name 'CTG.csv' --additional-options-csv-parsing '{"sep":"," , "header":0}' \
    --type-of-data-analysis-task 'classification' --target-variable-name 'NSP' \
    --data-preparations-options '{"ignore_low_variance":true, "remove_outliers":true, "remove_multicollinearity":true, "multicollinearity_threshold":0.7}' \
    --output-datasource-directory-to-be-mounted '/content' --output-datasource-file-name 'CTG_data-prep.csv'

In [2]:
%%writefile Dockerfile

FROM python:3.7
RUN python3 -m pip install pycaret==2.3.10
RUN python3 -m pip install Jinja2==3.1.2
RUN curl https://rclone.org/install.sh | bash

COPY data_preparation.py /tmp

Writing Dockerfile


Before commiting code to github, install github client (gh) by following instruction mentioned in https://github.com/cli/cli/blob/trunk/docs/install_linux.md

Use the colab's 'Terminal' icon present in left vertical pane to open linux terminal to type commands. Once 'gh' is installed, type **$gh auth login** (refer https://docs.github.com/en/get-started/getting-started-with-git/caching-your-github-credentials-in-git) to follow onscreen prompts. For colab, use **pasting of personal token** option. Personal tokens can be generated in https://github.com/settings/tokens

In [1]:
! git config --list

user.email=29448766+ShaswataJash@users.noreply.github.com
user.name=ShaswataJash
credential.https://github.com.helper=
credential.https://github.com.helper=!/usr/bin/gh auth git-credential
credential.https://gist.github.com.helper=
credential.https://gist.github.com.helper=!/usr/bin/gh auth git-credential


In [2]:
!rm -Rf kfpcomponent

In [3]:
!git clone https://github.com/ShaswataJash/kfpcomponent.git

Cloning into 'kfpcomponent'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [4]:
!mkdir kfpcomponent/DataPreparationUsingPycaret

In [5]:
!rsync -c data_preparation.py kfpcomponent/DataPreparationUsingPycaret/

In [6]:
!rsync -c Dockerfile kfpcomponent/DataPreparationUsingPycaret/

In [7]:
%cd kfpcomponent

/content/kfpcomponent


In [8]:
!git add -A

In [9]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	new file:   DataPreparationUsingPycaret/Dockerfile
	new file:   DataPreparationUsingPycaret/data_preparation.py



In [10]:
!git commit -a -m "my first commit"

[main 5913d45] my first commit
 2 files changed, 150 insertions(+)
 create mode 100644 DataPreparationUsingPycaret/Dockerfile
 create mode 100644 DataPreparationUsingPycaret/data_preparation.py


In [11]:
!git push origin main

Counting objects: 5, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 2.73 KiB | 2.73 MiB/s, done.
Total 5 (delta 0), reused 0 (delta 0)
To https://github.com/ShaswataJash/kfpcomponent.git
   ba1baf3..5913d45  main -> main
